In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
if not (os.path.exists('./data/userid-profile.tsv') and os.path.exists('./data/userid-timestamp-artid-artname-traid-traname.tsv')):
  raise ValueError('Los archivos userid-profile y userid-timestamp-artid-artname-traid-traname no fueron encontrados en el path')
else:
  print("Los archivos han sido cargados")

Los archivos han sido cargados


In [3]:
users=pd.read_csv('./data/userid-profile.tsv', sep = '\t', header=0 )

In [4]:
activity=pd.read_csv('./data/userid-timestamp-artid-artname-traid-traname.tsv', sep = '\t', names = [ 'userid', 'timestamp', 'musicbrainz-artist-id', 'artist-name', 'musicbrainz-track-id', 'track-name' ])

In [5]:
activity.head(10)

,userid,timestamp,musicbrainz-artist-id,artist-name,musicbrainz-track-id,track-name
0,user_000001,2009-05-04T23:08:57Z,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,2009-05-04T13:54:10Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15)
2,user_000001,2009-05-04T13:52:04Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc2 (Live_2009_4_15)
3,user_000001,2009-05-04T13:42:52Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Hibari (Live_2009_4_15)
4,user_000001,2009-05-04T13:42:11Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc1 (Live_2009_4_15)
5,user_000001,2009-05-04T13:38:31Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,To Stanford (Live_2009_4_15)
6,user_000001,2009-05-04T13:33:28Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Improvisation (Live_2009_4_15)
7,user_000001,2009-05-04T13:23:45Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Glacier (Live_2009_4_15)
8,user_000001,2009-05-04T13:19:22Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Parolibre (Live_2009_4_15)
9,user_000001,2009-05-04T13:13:38Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Bibo No Aozora (Live_2009_4_15)


### Primero removemos los 50 usuarios con menos interacciones para reducir el tamaño del dataframe

In [6]:
activity_lastFifty = activity.groupby('userid')['artist-name'].agg({'count'}).sort_values(by='count', ascending=False).tail(50)

In [7]:
activity_lastFifty_union = activity.merge(activity_lastFifty, how='inner', left_on='userid', right_index=True)
activity_lastFifty_union = activity_lastFifty_union[['userid', 'timestamp', 'musicbrainz-artist-id', 'artist-name', 'musicbrainz-track-id', 'track-name']]

In [8]:
dataframes = [activity,activity_lastFifty_union]
reduced_activity = pd.concat(dataframes).drop_duplicates(keep=False)

### Luego contamos las veces en las que un usuario escuchó un artista

In [9]:
user_artists_count = reduced_activity.groupby(['userid', 'artist-name']).count()

In [16]:
sorted_user_artists_count = user_artists_count['track-name'].reset_index(name='count').sort_values(['userid', 'count'], ascending=False)

### Por último sacamos un rating basado en la cantidad de veces que un artista fué escuchado de manera proporcional

In [17]:
# obtenemos un dataframe con el maximo de reproducciones por cada usuario
user_max = pd.DataFrame(sorted_user_artists_count.groupby(['userid'], sort=False)['count'].max()).reset_index()
user_max.columns = ['userid', 'max']

In [18]:
# hacemos merge con la tabla de usuarios y artistas
sorted_user_artists_count_max = sorted_user_artists_count.merge(user_max, how='left', left_on='userid', right_on='userid')

In [19]:
sorted_user_artists_count_max

,userid,artist-name,count,max
0,user_001000,Wilco,455,455
1,user_001000,Radiohead,444,455
2,user_001000,Animal Collective,432,455
3,user_001000,Girl Talk,417,455
4,user_001000,Aesop Rock,383,455
...,...,...,...,...
894494,user_000001,曽我部恵一,1,855
894495,user_000001,畠山美由紀,1,855
894496,user_000001,細野晴臣,1,855
894497,user_000001,青柳拓次,1,855


In [20]:
# creamos una nueva columna en el dataframe con el ranking que va de 0 a 5, donde 5 es el artista mas oido y 0 el menos oido
sorted_user_artists_count_max['rating'] = np.around((sorted_user_artists_count_max['count']/sorted_user_artists_count_max['max']) * 5, decimals=1)
sorted_user_artists_count_max

,userid,artist-name,count,max,rating
0,user_001000,Wilco,455,455,5.0
1,user_001000,Radiohead,444,455,4.9
2,user_001000,Animal Collective,432,455,4.7
3,user_001000,Girl Talk,417,455,4.6
4,user_001000,Aesop Rock,383,455,4.2
...,...,...,...,...,...
894494,user_000001,曽我部恵一,1,855,0.0
894495,user_000001,畠山美由紀,1,855,0.0
894496,user_000001,細野晴臣,1,855,0.0
894497,user_000001,青柳拓次,1,855,0.0


In [21]:
sorted_user_artists_count_max.to_csv('./data/preprocessed_user_item_rating.csv')

In [24]:
shuffled_ratings = sorted_user_artists_count_max.sample(frac=1).reset_index(drop=True)
shuffled_ratings

,userid,artist-name,count,max,rating
0,user_000872,Kt Tunstall,3,746,0.0
1,user_000327,Interpol,93,581,0.8
2,user_000706,The Police,2,484,0.0
3,user_000807,Shankar-Ehsaan-Loy,1,703,0.0
4,user_000223,Timo Räisänen,9,1985,0.0
...,...,...,...,...,...
894494,user_000172,Ocean'S Eleven Soundtrack,3,401,0.0
894495,user_000655,Pat Benatar,2,117,0.1
894496,user_000983,The Band,1,355,0.0
894497,user_000298,Outkast,48,7761,0.0


In [ ]:
sorted_user_artists_count_max.to_csv('./data/preprocessed_user_item_rating_shuffled.csv')

In [ ]:
# Partimos el dataset en 2 después haberle hecho shuffle

In [29]:
import math
num_half = math.floor((len(shuffled_ratings.index) - 1) / 2)
num_half

447249

In [30]:
first_half_ratings = shuffled_ratings.iloc[:, :num_half]
second_half_ratings = shuffled_ratings.iloc[:, num_half:]

In [31]:
first_half_ratings.to_csv('./data/preprocessed_user_item_rating_shuffled_first_half.csv')
second_half_ratings.to_csv('./data/preprocessed_user_item_rating_shuffled_second_half.csv')